# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 20 2022 8:43AM,237712,10,0085685119,ISDIN Corporation,Released
1,May 20 2022 8:42AM,237711,10,0085686586,ISDIN Corporation,Released
2,May 20 2022 8:42AM,237711,10,0085686587,ISDIN Corporation,Released
3,May 20 2022 8:42AM,237711,10,0085686593,ISDIN Corporation,Released
4,May 20 2022 8:42AM,237711,10,0085686596,ISDIN Corporation,Released
5,May 20 2022 8:42AM,237711,10,0085686600,ISDIN Corporation,Released
6,May 20 2022 8:42AM,237711,10,0085686602,ISDIN Corporation,Released
7,May 20 2022 8:42AM,237711,10,0085686607,ISDIN Corporation,Released
8,May 20 2022 8:42AM,237711,10,0085686701,ISDIN Corporation,Released
9,May 20 2022 8:42AM,237711,10,0085686720,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,237708,Released,4
23,237709,Released,1
24,237710,Released,1
25,237711,Released,18
26,237712,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237708,NaN,NaN,4.0
237709,NaN,NaN,1.0
237710,NaN,NaN,1.0
237711,NaN,NaN,18.0
237712,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3.0,0.0,2.0
237597,0.0,0.0,1.0
237598,0.0,0.0,1.0
237640,0.0,0.0,1.0
237643,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3,0,2
237597,0,0,1
237598,0,0,1
237640,0,0,1
237643,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,0,2
1,237597,0,0,1
2,237598,0,0,1
3,237640,0,0,1
4,237643,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,,2
1,237597,,,1
2,237598,,,1
3,237640,,,1
4,237643,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation
19,May 20 2022 8:38AM,237710,19,ACG North America LLC
20,May 20 2022 8:33AM,237709,10,Bio-PRF
21,May 20 2022 8:33AM,237708,10,Emerginnova
25,May 20 2022 7:57AM,237707,19,"Emersa Waterbox, LLC"
26,May 20 2022 7:51AM,237706,19,"AdvaGen Pharma, Ltd"
35,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC."
36,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions
37,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation,,,1
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation,,,18
2,May 20 2022 8:38AM,237710,19,ACG North America LLC,,,1
3,May 20 2022 8:33AM,237709,10,Bio-PRF,,,1
4,May 20 2022 8:33AM,237708,10,Emerginnova,,,4
5,May 20 2022 7:57AM,237707,19,"Emersa Waterbox, LLC",,,1
6,May 20 2022 7:51AM,237706,19,"AdvaGen Pharma, Ltd",,,9
7,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",,,1
8,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,,,1
9,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation,1,,
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation,18,,
2,May 20 2022 8:38AM,237710,19,ACG North America LLC,1,,
3,May 20 2022 8:33AM,237709,10,Bio-PRF,1,,
4,May 20 2022 8:33AM,237708,10,Emerginnova,4,,
5,May 20 2022 7:57AM,237707,19,"Emersa Waterbox, LLC",1,,
6,May 20 2022 7:51AM,237706,19,"AdvaGen Pharma, Ltd",9,,
7,May 20 2022 12:00AM,237704,10,"Herbivore Botanicals, LLC.",1,,
8,May 20 2022 12:00AM,237693,10,Acute Outpatient Solutions,1,,
9,May 20 2022 12:00AM,237694,10,Acute Outpatient Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation,1,,
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation,18,,
2,May 20 2022 8:38AM,237710,19,ACG North America LLC,1,,
3,May 20 2022 8:33AM,237709,10,Bio-PRF,1,,
4,May 20 2022 8:33AM,237708,10,Emerginnova,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation,1.0,NaN,NaN
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation,18.0,NaN,NaN
2,May 20 2022 8:38AM,237710,19,ACG North America LLC,1.0,NaN,NaN
3,May 20 2022 8:33AM,237709,10,Bio-PRF,1.0,NaN,NaN
4,May 20 2022 8:33AM,237708,10,Emerginnova,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 20 2022 8:43AM,237712,10,ISDIN Corporation,1.0,0.0,0.0
1,May 20 2022 8:42AM,237711,10,ISDIN Corporation,18.0,0.0,0.0
2,May 20 2022 8:38AM,237710,19,ACG North America LLC,1.0,0.0,0.0
3,May 20 2022 8:33AM,237709,10,Bio-PRF,1.0,0.0,0.0
4,May 20 2022 8:33AM,237708,10,Emerginnova,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2614642,57.0,4.0,0.0
17,237654,0.0,1.0,0.0
18,950538,4.0,0.0,0.0
19,950821,16.0,0.0,0.0
20,712874,22.0,0.0,3.0
21,237703,1.0,0.0,0.0
22,237640,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2614642,57.0,4.0,0.0
1,17,237654,0.0,1.0,0.0
2,18,950538,4.0,0.0,0.0
3,19,950821,16.0,0.0,0.0
4,20,712874,22.0,0.0,3.0
5,21,237703,1.0,0.0,0.0
6,22,237640,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,4.0,0.0
1,17,0.0,1.0,0.0
2,18,4.0,0.0,0.0
3,19,16.0,0.0,0.0
4,20,22.0,0.0,3.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,17,Released,0.0
2,18,Released,4.0
3,19,Released,16.0
4,20,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,17,18,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Executing,4.0,1.0,0.0,0.0,0.0,0.0,0.0
Released,57.0,0.0,4.0,16.0,22.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,57.0,0.0,4.0,16.0,22.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Released,57.0,0.0,4.0,16.0,22.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()